In [ ]:
import numpy as np
import torch
from scipy.integrate import odeint
import torch.nn as nn
import matplotlib.pyplot as plt
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from torch import nn, autograd
from src.transformer_nn import TransformerModel
#from src.transformer_nn import SimpleLinearModel
from src.duffing_generator_batch import DuffingGeneratorClass
from src.physics_loss import physics_loss_class
import torch.optim as op 
%load_ext jupyter_ai
import torch.optim as optim
#from torch.utils.data import Data
import transformers #import Transformer


In [ ]:
x = torch.linspace(0, 1, 1000).view(-1, 1)
dg = DuffingGeneratorClass()  # Instantiate your class

device = 'cuda'
torch.manual_seed(123)
physics_loss_instance = physics_loss_class()
# Set up the physics loss training locations
x_physics = torch.linspace(0, 1, 15).view(-1, 1).requires_grad_(True)
X_BOUNDARY = 0.0  # Example value, adjust as needed
F_BOUNDARY = 0.0  # Example value, adjust as needed
N_INPUT= 6 #(or more, depending on your features)
N_OUTPUT= 1 #(or the number of features you want to predict)
N_HIDDEN= 256 # or 256
N_LAYERS= 4 #to 4
N_HEADS= 8 #or 8 (should divide evenly into N_HIDDEN)

# Initialize model, optimizer, and loss function
model = TransformerModel(N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS, N_HEADS).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-5)
loss_instance = physics_loss_class()

num_epochs = 10  # Adjust as needed

In [ ]:
'''# Create a small batch of test data with varying time
test_time = torch.linspace(0, 1, 10).view(-1, 1)  # 10 time steps
test_params = torch.zeros(10, 5)  # Keep other parameters constant
test_x_combined = torch.cat((test_time, test_params), dim=1).to(device)

# Get model's predictions for this test data
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    test_output = model(test_x_combined)
    print("Model output for test data:", test_output)'''

In [ ]:

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch_idx, batch in enumerate(dg.duffing_generator()):
        x_combined, y_batch = batch
        x_combined, y_batch = x_combined.to(device), y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(x_combined)
        
        # Compute physics loss
        loss_physics = loss_instance.physics_loss(model, batch, x_combined, y_batch)
        

        # Combine losses
        total_loss = loss_physics 
        
        total_loss.backward()
        optimizer.step()
    
        # Optionally print or log the loss
        print(f"Batch {batch_idx}, Total Loss: {total_loss.item()}")

    # Visualization every 10 epochs
    if (epoch + 1) % 10 == 0:
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():
            yh = model(x_combined).detach().cpu().numpy()
            time_points = x_combined[:, 0].cpu().numpy()  # Extract time points from combined input
            plt.figure(figsize=(10, 5))
            plt.plot(time_points, y_batch.cpu().numpy(), label='Ground Truth')
            plt.plot(time_points, yh, label='Neural Network Output')
            plt.scatter(time_points, y_batch.cpu().numpy(), color='red', label='Training Points')
            plt.xlabel("Time")
            plt.ylabel("Value")
            plt.legend()
            plt.title(f"Model Output vs Ground Truth at Epoch {epoch+1}")
            plt.show()

    # Optional: Add validation logic here

# Optional: Save the model after training
# torch.save(model.state_dict(), 'model.pth')

In [ ]:
print(torch.cuda.is_available())

In [ ]:
# Define the dimensions or use actual test data for test_input
test_input_dimensions = (100, 6)  # Example dimensions, modify as necessary
test_input = torch.randn(test_input_dimensions, dtype=torch.double, requires_grad=True)  # Create a random tensor with requires_grad set to True

# Convert test input to Float
test_input = test_input.float()

# Move model and test_input to the same device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
test_input = test_input.to(device)




# Test function for gradcheck
def test_func(input):
    return model(input)
# Perform the gradcheck with the modified test input
gradcheck_passed = gradcheck(test_func, test_input, eps=1e-5, atol=1e-4)
print("Gradcheck passed:", gradcheck_passed)

In [ ]:

# Create a sample input tensor with batch size greater than 1
sample_input = torch.randn(2, N_INPUT, requires_grad=True)

# Move model and sample_input to the same device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
sample_input = sample_input.to(device)

# Forward pass
output = model(sample_input)

# Sample loss computation (replace with a loss appropriate for your model)
sample_loss = output.sum()

# Backward pass
sample_loss.backward()

# Check if gradients are computed for the input
print("Gradients for the input tensor:", sample_input.grad)